In [9]:
# !pip install -U langchain-openai

In [11]:
uv add langchain

/Users/aditya16902/Desktop/Github/awesome-llm-apps/venv/bin/python: No module named uv
Note: you may need to restart the kernel to use updated packages.


In [8]:
uv add pip -U composio-langchain

/Users/aditya16902/Desktop/Github/awesome-llm-apps/venv/bin/python: No module named uv
Note: you may need to restart the kernel to use updated packages.


In [40]:
import os
import getpass
from composio import Composio
from composio_langchain import LangchainProvider
from deepagents import create_deep_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langgraph.checkpoint.memory import MemorySaver

if not os.environ.get("COMPOSIO_API_KEY"):
    os.environ["COMPOSIO_API_KEY"] = getpass.getpass("Enter your Composio API key: ")

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")


##Tools
# Text to list converter (Converts RQs into list)
@tool
def generate_questions_list(q: str) -> list:
    """Takes string q, with questions about the given topic in the specified domain. 
    Its formatted as a numbered list, and NOTHING ELSE. Then this function/ tool helps to Extract questions into a list."""

    questions_list = [question.strip() for question in q.split('\n') if question.strip()]
    return questions_list

#Tools from Composio for agent centric web-search (Tavily) and research (Perplexity) and document creation (Google docs)
composio = Composio(provider=LangchainProvider())
composio_tools = composio.tools.get(user_id="default", toolkits=["COMPOSIO_SEARCH_TAVILY_SEARCH", "PERPLEXITYAI_PERPLEXITY_AI_SEARCH", "GOOGLEDOCS_CREATE_DOCUMENT_MARKDOWN"])


# Child Agent 1 [Generates Research Questions]
research_subagent_1 = {
    "name": "research-question-agent",
    "description": "Used to generate research questions about the topic '{topic}' in the domain '{domain}'",
    "system_prompt": """
        You are an expert at breaking down research topics into specific questions.
        """,
    "tools": [*composio_tools, generate_questions_list]

}

# Child Agent 2 [research a specific question]
research_subagent_2 = {
    "name": "research-question-agent",
    "description": "Used to answer a research question",
    "system_prompt": """
        You are a sophisticated research assistant, skilled in researching about a particular research question related to a given domain
        """,
    "tools": [*composio_tools]
}

# Child Agent 3 [Compiling final report]
research_subagent_3 = {
    "name": "report-compiling-agent",
    "description": "Used to create professional, McKinsey-style report.",
    "system_prompt": f"""
            You are a sophisticated research assistant. skilled in creating research reports from existing research findings.
            """,
    "tools": [*composio_tools]
    
}



# mother agent
research_subagents = [research_subagent_1, research_subagent_2, research_subagent_3]
model = init_chat_model(model="openai:gpt-5")
checkpointer = MemorySaver()
research_instructions1 = f"""You are a sophisticated head researcher. 

        1) Based on the users input ( - Topic: topic, Domain: domain), Generate exactly 5 specific yes/no research questions 
        about the given topic (use COMPOSIO_SEARCH_TAVILY_SEARCH, PERPLEXITYAI_PERPLEXITY_AI_SEARCH if needed) in the specified domain.
        Respond ONLY with the text of the 5 questions formatted as a numbered list, and NOTHING ELSE. 
        Then use generate_questions_list tool to Extract questions into a list.
        
        2)Now, Answer the each of the research questions about the topic 'topic' in the domain 'domain':\n\nquestion\n\nUse 
        the PERPLEXITYAI_PERPLEXITY_AI_SEARCH and COMPOSIO_SEARCH_TAVILY_SEARCH tools to provide a concise, well-sourced answer.

        3)Now, Compile the following research findings into a professional, McKinsey-style report. The report should be structured as follows:

            1. Executive Summary/Introduction: Briefly introduce the topic and domain, and summarize the key findings.
            2. Research Analysis: For each research question, create a section with a clear heading and provide a detailed, analytical answer. Do NOT use a Q&A format; instead, weave the answer into a narrative and analytical style.
            3. Conclusion/Implications: Summarize the overall insights and implications of the research.

            Use clear, structured HTML for the report.

            Topic: topic
            Domain: domain

            Research Questions and Findings (for your reference):
            qa_sections

            Use the GOOGLEDOCS_CREATE_DOCUMENT_MARKDOWN tool to create a Google Doc with the report. The text should be in HTML format. You have to create the google document with all the compiled info.

        !return output format ONLY as a dictionary, and NOTHING ELSE. with key being the research Questions, final report being keys and values being your Research Finding for respective RQs and Research report content (Use HTML content inside accordingly, so i can directly parse later accordingly on website).
 

        """
research_instructions2 = f"""
You are a senior, highly analytical head researcher.

Your task proceeds in **three strict phases**. Follow each phase exactly, and adhere to all formatting requirements.

---

### **PHASE 1 — Generate Research Questions**

1. Based on the user's input (**Topic: topic**, **Domain: domain**), generate **exactly 5 highly specific YES/NO research questions**.
2. These questions must be evaluative, research-ready, and directly answerable using available search tools  
   (**COMPOSIO_SEARCH_TAVILY_SEARCH**, **PERPLEXITYAI_PERPLEXITY_AI_SEARCH**).
3. Respond **ONLY** with the text of the 5 questions, formatted as a **numbered list (1–5)** and **nothing else**.
4. After producing the list, call the **generate_questions_list** tool to extract the questions into a list.

---

### **PHASE 2 — make a deep research about the Research Questions**

For each generated question:

1. Use **PERPLEXITYAI_PERPLEXITY_AI_SEARCH** and **COMPOSIO_SEARCH_TAVILY_SEARCH** to gather evidence.
2. Produce a **detiled, well-sourced analytical answer**.
3. Structure the output as key–value pairs where keys are the research questions and values are the answers (this will later be compiled).

---

### **PHASE 3 — Compile a McKinsey-Style Research Report**

Using the findings from Phase 2, compile a polished, consulting-grade report with the following structure. The report should be comprehensive and detailed—matching the depth and length typically produced by advanced LLM “deep research” workflows.

1. **Executive Summary / Introduction**  
   Introduce the topic and domain and summarize the key insights.

2. **Research Analysis**  
   For each research question, create a section with a clear subheading.  
   **Do NOT use a Q&A format.**  
   Instead, integrate the findings into a coherent analytical narrative for each section.

3. **Conclusion & Implications**  
   Summarize cross-cutting insights and outline implications for decision-makers.

Requirements:

- The entire report must be formatted in **clean, structured HTML**.
- After compiling the report, call **GOOGLEDOCS_CREATE_DOCUMENT_MARKDOWN** to create a Google Doc containing the full HTML content.

---

### **FINAL OUTPUT CONSTRAINTS**

Return the final output **ONLY as a HTML**, with no surrounding text.

Use this structure exactly so it can be directly parsed by the website.
"""



agent = create_deep_agent(model=model,subagents = research_subagents,
                            system_prompt = research_instructions2, tools=[generate_questions_list, *composio_tools],     
                            interrupt_on = {"generate_questions_list": {"allowed_decisions": ["approve", "edit"]}},
                            checkpointer=checkpointer)


In [42]:
config = {"configurable": {"thread_id": "1-accept"}}
result = agent.invoke({
    "messages": [{"role": "user", "content": "Topic: History of Reinforcement Learning, Domain:Technology"}]
}, config=config)

In [43]:
# Check if execution was interrupted
if result.get("__interrupt__"):
    # Extract interrupt information
    interrupts = result["__interrupt__"][0].value
    action_requests = interrupts["action_requests"]
    review_configs = interrupts["review_configs"]

    # Create a lookup map from tool name to review config
    config_map = {cfg["action_name"]: cfg for cfg in review_configs}

    # Display the pending actions to the user
    for action in action_requests:
        review_config = config_map[action["name"]]
        print(f"Tool: {action['name']}")
        print(f"Arguments: {action['args']}")
        print(f"Allowed decisions: {review_config['allowed_decisions']}")

Tool: generate_questions_list
Arguments: {'q': '1. Did Sutton (1988) introduce temporal-difference learning TD(λ) as a general method distinct from dynamic programming and Monte Carlo?\n2. Did Watkins and Dayan (1992) prove that tabular Q-learning converges to the optimal action-value function Q* for finite MDPs under standard conditions?\n3. Did Lin (1992) introduce experience replay prior to its popularization in DeepMind’s DQN (2015)?\n4. Did Tesauro’s TD-Gammon (1992–1995) achieve strong backgammon play primarily through self-play reinforcement learning without supervised expert data?\n5. Did AlphaGo (2016) train its initial policy network purely via reinforcement learning without supervised pretraining?'}
Allowed decisions: ['approve', 'edit']


Accept


In [44]:
from langgraph.types import Command


# Check if execution was interrupted
if result.get("__interrupt__"):

    # Get user decisions (one per action_request, in order)
    decisions = [
        {"type": "approve"}  # User approved the deletion
    ]

    # Resume execution with decisions
    result = agent.invoke(
        Command(resume={"decisions": decisions}),
        config=config  # Must use the same config!
    )

# Process final result
print(result["messages"][-1].content)

<html>
  <head>
    <meta charset="utf-8">
    <title>History of Reinforcement Learning — Technology Domain</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
      body { font-family: Arial, Helvetica, sans-serif; line-height: 1.5; color: #222; margin: 0; padding: 0; }
      header, section, footer { max-width: 960px; margin: 0 auto; padding: 24px; }
      header { background: #0b3d60; color: white; }
      h1 { margin: 0 0 8px 0; font-size: 28px; }
      h2 { margin-top: 32px; font-size: 22px; color: #0b3d60; }
      h3 { margin-top: 20px; font-size: 18px; color: #0b3d60; }
      a { color: #0b3d60; text-decoration: none; border-bottom: 1px solid #c7d8e8; }
      a:hover { border-bottom-color: #0b3d60; }
      ul { margin-top: 8px; }
      blockquote { margin: 12px 0; padding: 8px 12px; background: #f7f9fb; border-left: 3px solid #c7d8e8; }
      .small { font-size: 90%; color: #555; }
      .ref { font-size: 90%; }
      .tag { display: inli

edit

In [ ]:
from langgraph.types import Command


# Check if execution was interrupted
if result.get("__interrupt__"):

    # User decides to edit the recipient
    decisions = [{
        "type": "edit",
        "edited_action": {
            "name": action['name'],  # Must include the tool name
            "args": {"q": "1. Did Sutton (1988) introduce temporal-difference learning TD(λ) as a general method distinct from dynamic programming and Monte Carlo?\n2. Did Watkins and Dayan (1992) prove that tabular Q-learning converges to the optimal action-value function Q* for finite MDPs under standard conditions?\n3. Did Lin (1992) introduce experience replay prior to its popularization in DeepMind’s DQN (2015)?\n4. Did Tesauro’s TD-Gammon (1992–1995) achieve strong backgammon play primarily through self-play reinforcement learning without supervised expert data?"}
        }
    }]

    result = agent.invoke(
        Command(resume={"decisions": decisions}),
        config=config
    )

# Process final result
print(result["messages"][-1].content)

In [46]:
html_content = result["messages"][-1].content  # this is your HTML string

with open("temp.html", "w", encoding="utf-8") as f:
    f.write(html_content)

In [51]:
html_content = result["messages"][-1].content

In [67]:
import http.server
import socketserver
import webbrowser
import threading
import time

PORT = 8000
DIRECTORY = "."  # Current directory

def open_browser():
    time.sleep(1)
    webbrowser.open(f'http://localhost:{PORT}/temp.html')

Handler = http.server.SimpleHTTPRequestHandler
with socketserver.TCPServer(("", PORT), Handler) as httpd:
    # Auto-open browser
    browser_thread = threading.Thread(target=open_browser)
    browser_thread.daemon = True
    browser_thread.start()
    
    print(f"Serving {DIRECTORY} at http://localhost:{PORT}/temp.html")
    print("Press Ctrl+C to stop")
    httpd.serve_forever()


Serving . at http://localhost:8000/temp.html
Press Ctrl+C to stop


127.0.0.1 - - [05/Dec/2025 12:32:06] "GET /temp.html HTTP/1.1" 200 -


KeyboardInterrupt: 

In [19]:
result

{'messages': [HumanMessage(content='Topic: History of Reinforcement Learning, Domain:Technology', additional_kwargs={}, response_metadata={}, id='d5e794ca-718f-4db5-ac10-d83f8565a3c5'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3899, 'prompt_tokens': 4992, 'total_tokens': 8891, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 3712, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CjCmZf4H9VXhe3ESFuMyh6RLFLAe5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019aeba3-2b05-7cf2-9314-3f8aca93d089-0', tool_calls=[{'name': 'generate_questions_list', 'args': {'q': '1. Did Richard Bellman’s dynamic programming (1957) directly inform the bootstrapping principle used in te

In [55]:
result

{'messages': [HumanMessage(content='Topic: History of Reinforcement Learning, Domain:Technology', additional_kwargs={}, response_metadata={}, id='91a2b498-285d-4380-866e-bd89276e5967'),
  AIMessage(content='1. Did Sutton (1988) introduce temporal-difference learning TD(λ) as a general method distinct from dynamic programming and Monte Carlo?\n2. Did Watkins and Dayan (1992) prove that tabular Q-learning converges to the optimal action-value function Q* for finite MDPs under standard conditions?\n3. Did Lin (1992) introduce experience replay prior to its popularization in DeepMind’s DQN (2015)?\n4. Did Tesauro’s TD-Gammon (1992–1995) achieve strong backgammon play primarily through self-play reinforcement learning without supervised expert data?\n5. Did AlphaGo (2016) train its initial policy network purely via reinforcement learning without supervised pretraining?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2763, 'prompt_tokens': 5084,

## Streamlit

In [59]:
# !pip install streamlit

In [63]:
# ===== ADD YOUR AGENT HERE =====

st.session_state.agent = agent
config = {"configurable": {"thread_id": "1-accept"}}
# ===============================


2025-12-05 10:23:37.784 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-05 10:23:37.785 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


## old

In [3]:
result

{'messages': [HumanMessage(content='Topic: History of Reinforcement Learning, Domain:Technology', additional_kwargs={}, response_metadata={}, id='a13ac97f-fde0-4bc0-a7d1-70969fae1442'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 4908, 'total_tokens': 4955, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-CjC4AFeM3FL1WwDEeShRvZvQ2NB0a', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019aeb79-2d1e-7c42-8eea-c9d94ccbca9a-0', tool_calls=[{'name': 'task', 'args': {'description': 'Generate exactly 5 specific yes/no research questions about the history of Reinforcement Learning in the tec

In [4]:
# Check if execution was interrupted
if result.get("__interrupt__"):
    # Extract interrupt information
    interrupts = result["__interrupt__"][0].value
    action_requests = interrupts["action_requests"]
    review_configs = interrupts["review_configs"]

    # Create a lookup map from tool name to review config
    config_map = {cfg["action_name"]: cfg for cfg in review_configs}

    # Display the pending actions to the user
    for action in action_requests:
        review_config = config_map[action["name"]]
        print(f"Tool: {action['name']}")
        print(f"Arguments: {action['args']}")
        print(f"Allowed decisions: {review_config['allowed_decisions']}")

Tool: generate_questions_list
Arguments: {'q': '1. Did the concept of Reinforcement Learning originate before the advent of digital computers?\n2. Was the development of Q-learning a pivotal moment in the history of Reinforcement Learning?\n3. Has Reinforcement Learning been predominantly developed by academic institutions compared to commercial enterprises?\n4. Did the introduction of Deep Reinforcement Learning techniques significantly boost performance in complex gaming environments, such as Go?\n5. Was Reinforcement Learning initially applied effectively in robotics before it found applications in other areas, such as finance or healthcare?'}
Allowed decisions: ['approve', 'edit']


In [6]:
from langgraph.types import Command


# Check if execution was interrupted
if result.get("__interrupt__"):

    # Get user decisions (one per action_request, in order)
    decisions = [
        {"type": "approve"}  # User approved the deletion
    ]

    # Resume execution with decisions
    result = agent.invoke(
        Command(resume={"decisions": decisions}),
        config=config  # Must use the same config!
    )

# Process final result
print(result["messages"][-1].content)

I have compiled the research findings into a professional, McKinsey-style report. Here is a summary of the key sections and findings:

### Research Questions and Findings

1. **Did the concept of Reinforcement Learning originate before the advent of digital computers?**
   - The concept has roots in behavioral psychology but was formalized with digital computational methods.

2. **Was the development of Q-learning a pivotal moment in the history of Reinforcement Learning?**
   - Yes, it introduced model-free approaches, off-policy learning, and underpinned deep Q-network advancements.

3. **Has Reinforcement Learning been predominantly developed by academic institutions compared to commercial enterprises?**
   - Initial developments were academic, but commercial enterprises like Google DeepMind have accelerated recent advancements.

4. **Did the introduction of Deep Reinforcement Learning techniques significantly boost performance in complex gaming environments, such as Go?**
   - Abso

In [10]:
print(result["messages"][-3].content)

In [8]:
result

{'messages': [HumanMessage(content='Topic: History of Reinforcement Learning, Domain:Technology', additional_kwargs={}, response_metadata={}, id='a13ac97f-fde0-4bc0-a7d1-70969fae1442'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 4908, 'total_tokens': 4955, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-CjC4AFeM3FL1WwDEeShRvZvQ2NB0a', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019aeb79-2d1e-7c42-8eea-c9d94ccbca9a-0', tool_calls=[{'name': 'task', 'args': {'description': 'Generate exactly 5 specific yes/no research questions about the history of Reinforcement Learning in the tec

In [ ]:

for s in agent.stream({"messages": [{"role": "user", "content": "Topic: History of Reinforcement Learning, Domain:Technology"}]}, config=config):
    print(s)

{'PatchToolCallsMiddleware.before_agent': {'messages': Overwrite(value=[HumanMessage(content='Topic: History of Reinforcement Learning, Domain:Technology', additional_kwargs={}, response_metadata={}, id='31e81f4a-842d-47d3-a643-1b33b3eaaa9f'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 4908, 'total_tokens': 4952, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 3840}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e819e3438b', 'id': 'chatcmpl-CjBGSYV8bMKbPVMIwsuITiSTOC1pG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019aeb4a-21e4-7bd2-a71b-27bbd8b9f925-0', tool_calls=[{'name': 'task', 'args': {'description': "Generate exactly 5 specific yes/no research quest

In [ ]:
for s in agent.stream(Command(resume=[{"type": "edit", "args": {"action": "generate_questions_list", "args": args}}]), config=config):
    print(s)

In [22]:
# If this calls a tool with an interrupt, this will then return an interrupt
# Replace the `...` with the tool name you want to call, and the arguments

args = {'q': '2. Was the first practical application of reinforcement learning in technology developed in the 1990s?\n3. Was the development of the Q-Learning algorithm in the 1980s a pivotal moment in the history of reinforcement learning?\n4. Have deep reinforcement learning techniques primarily emerged within the last two decades?\n5. Did the release of the AlphaGo program by DeepMind in 2016 significantly advance the public and academic interest in reinforcement learning technologies?'}
for s in agent.stream(Command(resume=[{"type": "edit", "args": {"action": "generate_questions_list", "args": args}}]), config=config):
    print(s)

TypeError: list indices must be integers or slices, not str

TypeError: list indices must be integers or slices, not str

In [17]:
s

{'__interrupt__': (Interrupt(value={'action_requests': [{'name': 'generate_questions_list', 'args': {'q': '1. Was the concept of Reinforcement Learning introduced before the 1980s?\n2. Did the development of Q-Learning occur in the late 20th century?\n3. Was Reinforcement Learning significantly influenced by the principles of behavioral psychology?\n4. Did early applications of Reinforcement Learning technologies face computational limitations due to hardware constraints?\n5. Was the development of Deep Q-Networks (DQN) pivotal in advancing the practical applications of Reinforcement Learning?'}, 'description': "Tool execution requires approval\n\nTool: generate_questions_list\nArgs: {'q': '1. Was the concept of Reinforcement Learning introduced before the 1980s?\\n2. Did the development of Q-Learning occur in the late 20th century?\\n3. Was Reinforcement Learning significantly influenced by the principles of behavioral psychology?\\n4. Did early applications of Reinforcement Learning 

In [ ]:
import uuid

config = {"configurable": {"thread_id": str(uuid.uuid4())}}

result = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Topic: History of Reinforcement Learning, Domain:Technology"
    }]
}, config=config)




KeyboardInterrupt: 